In [46]:
# 성능 향상이 막혔다면 다시 EDA를 해라
# 일반쓰레기, 철, 종이, 종이팩
# => 하나에 대해 분석, 수정 후 학습해보고 성능이 향상되는가 확인하기

In [47]:
from tools import *

In [64]:
cfg = EasyDict(
    target = 'General trash',
    anno_path = '../../dataset/fold_1/stratified_train.json',
    output_path = './output/statistics.txt',
    classes = ["General trash", "Paper", "Paper pack", "Metal", "Glass", 
            "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing"],
    hist_width = 1000,
    hist_height = 400
)
df,dct = load(cfg)
t_df = df[df.class_name == cfg.target]
sample(t_df, 3), df.shape, t_df.shape

loading annotations into memory...
Done (t=0.08s)
creating index...
index created!


,image_id,image_name,class_name,class_id,x_min,y_min,x_max,y_max
4809,1274,train/1274.jpg,General trash,0,348.0,419.0,699.0,746.0
8438,2252,train/2252.jpg,General trash,0,465.0,653.0,684.0,833.0
6279,1667,train/1667.jpg,General trash,0,335.0,286.0,716.0,838.0


(None, (18281, 8), (3312, 8))

In [65]:
hist_ann_per_img(t_df, cfg)

t_bbox_df = convert_to_bbox(t_df)
box_bbox_area(t_bbox_df, cfg)
box_aspect_ratio(t_bbox_df, cfg)

cal_statisitcs(t_df, t_bbox_df, cfg, single=True)

In [66]:
def filter(series, value, type='eq'):
    if type == 'eq': return (series == value).values
    if type == 'gt': return (series > value).values
    if type == 'get': return (series >= value).values
    if type == 'lt': return (series < value).values
    if type == 'let': return (series <= value).values
    if type == 'neq': return (series != value).values
    if type == 'in': return (series.apply(lambda x: x in value)).values

def apply_filter(df, filter):
    return df[filter]

count_upper_limit = 15
t_count_df = t_df.groupby('image_id').count().reset_index()
filter_count = filter(
    t_df.image_id,
    apply_filter(
        t_count_df, 
        filter(
            t_df.groupby('image_id').count().reset_index().iloc[:,1], 
            count_upper_limit, 
            'let')).image_id.values,
    'in'
)
fixed_t_df = apply_filter(t_df, filter_count)
fixed_t_bbox_df = apply_filter(t_bbox_df, filter_count)

bbox_upper_limit = 333081 /1024/1024
filter_area = filter(fixed_t_bbox_df.frac_bbox_area, bbox_upper_limit, 'let')
fixed_t_df = apply_filter(fixed_t_df, filter_area)
fixed_t_bbox_df = apply_filter(fixed_t_bbox_df, filter_area)

bbox_upper_ratio = 2.82
filter_ratio = filter(fixed_t_bbox_df.aspect_ratio, bbox_upper_ratio, 'let')
fixed_t_df = apply_filter(fixed_t_df, filter_ratio)
fixed_t_bbox_df = apply_filter(fixed_t_bbox_df, filter_ratio)

In [67]:
hist_ann_per_img(fixed_t_df, cfg)

box_bbox_area(fixed_t_bbox_df, cfg)
box_aspect_ratio(fixed_t_bbox_df, cfg)

cal_statisitcs(fixed_t_df, fixed_t_bbox_df, cfg, single=True)

In [72]:
from copy import deepcopy

fixed_dct = deepcopy(dct)
print(fixed_dct.keys())

fixed_dct['images'] = [
    x for x in fixed_dct['images'] 
    if x['id'] not in t_df.image_id.values or x['file_name'] in fixed_t_df.image_name.values]
fixed_dct['annotations'] = [
    x for x in fixed_dct['annotations'] 
    if x['image_id'] not in t_df.image_id.values or  x['image_id'] in fixed_t_df.image_id.values]
len(fixed_dct['images']),'<==',len(dct['images']), len(fixed_dct['annotations']),'<==',len(dct['annotations'])

dict_keys(['info', 'licenses', 'images', 'categories', 'annotations'])


(3706, '<==', 3908, 17438, '<==', 18281)

In [70]:
save_json('output/fixed_train.json', fixed_dct)